## Library imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from chromatography import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt

## Dataset Import

In [3]:
alist = pd.read_csv('../data/Roca.csv')

## Useful functions definition

In [4]:
def step_decay(lr, iteration, num_episodes, steps=10, decay_factor=1.5):
    if iteration % (num_episodes // steps) == 0:
        return lr / decay_factor
    
    return lr

In [5]:
def loss_field(exp, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], [0.25, 3000000])
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [12]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=None)
pol = Policy(2)
losses, best_program, mus, sigmas, n_pars = reinforce(
        exp, 
        pol, 
        [.25, 33.25], 
        num_episodes=3000, 
        batch_size=30,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=2),
        lr=.1, 
        print_every=50,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=1.2),
        baseline=0.5
    )

Loss: 1.746980851045825, epoch: 50/3000
Loss: 1.716374707881857, epoch: 100/3000
Loss: 1.6597573468068367, epoch: 150/3000
Loss: 1.3737389641281434, epoch: 200/3000
Loss: 1.092781948586603, epoch: 250/3000
Loss: 0.9939358965688538, epoch: 300/3000
Loss: 0.888126129294204, epoch: 350/3000
Loss: 0.8263639242687576, epoch: 400/3000
Loss: 0.7811212753875661, epoch: 450/3000
Loss: 0.7293571459027924, epoch: 500/3000
Loss: 0.6860798181171397, epoch: 550/3000
Loss: 0.6499567060487806, epoch: 600/3000
Loss: 0.6192131304872321, epoch: 650/3000
Loss: 0.5965960466791934, epoch: 700/3000
Loss: 0.5724542318956334, epoch: 750/3000
Loss: 0.56484819481644, epoch: 800/3000
Loss: 0.5470365052201063, epoch: 850/3000
Loss: 0.5417000259471177, epoch: 900/3000
Loss: 0.5339020550264203, epoch: 950/3000
Loss: 0.53043602543816, epoch: 1000/3000
Loss: 0.525334900178863, epoch: 1050/3000
Loss: 0.5194614993302298, epoch: 1100/3000
Loss: 0.5097636177569084, epoch: 1150/3000
Loss: 0.5070234854270702, epoch: 1200/30

In [16]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), [0.25, 3000000])
exp.print_analytes(rc=(7, 5))
exp.loss()

0.357462037473586

In [8]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
plt.legend()
plt.show()

In [9]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
plt.legend()
plt.show()

In [ ]:
plt.plot(losses)

In [15]:
exp.reset()
exp.run_all(best_program.numpy(), [.25, 33.25])
exp.print_analytes(rc=(7, 5), title="Best Result")
exp.loss()

0.35164982424833197

In [ ]:
#Y[np.unravel_index(np.argmin(Loss_field), Loss_field.shape)]

In [10]:
X, Y, Loss_field = loss_field(exp, N = 300)

In [13]:
fig, ax = plt.subplots()
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()
line, = plt.plot(mus[:, 0], mus[:, 1], color='k')

def update(num, x, y, line):
    line.set_data(x[num-25:num], y[num-25:num])
    line.axes.axis([0, 1, 0, 1])
    return line,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus[:, 0], mus[:, 1], line],
                              interval=1, blit=False)
plt.show()

In [ ]:
from mayavi import mlab
mlab.surf(X.T[0:50, 0:50], Y.T[0:50, 0:50], Loss_field[0:50, 0:50])

In [ ]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=None)
pol = PolicyTime(2)
losses, best_program, mus, sigmas, n_par = reinforce_delta_tau(
        exp, 
        pol,
        num_episodes=5000, 
        batch_size=10,
        optim=torch.optim.SGD,
        lr=.2, 
        print_every=200,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=1.05),
        baseline=0.15,
    )
exp.print_analytes(rc=(7, 5))

In [ ]:
exp.reset()
exp.run_all(best_program[0], best_program[1])
exp.print_analytes(rc=(7, 5))
exp.loss()

In [ ]:
plt.plot(losses)

In [ ]:
plt.plot(mus[:, 0], label='Mu: 0')
plt.plot(mus[:, 1], label='Mu: 1')
plt.plot(mus[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: 0')
plt.plot(sigmas[:, 1], label='Sigma: 1')
plt.plot(sigmas[:, 2], label='Sigma: 2')
plt.legend()
plt.show()